**Import statements**


In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import constants
import functions3 as functions

%matplotlib inline

**Load Data**

In [2]:
all_static_data = functions.load_static_sensors_calibrated()
all_static_data = functions.transform_to_grid_coordinates(all_static_data)
mobile_sensor_data = functions.load_mobile_sensors()
mobile_sensor_data = functions.transform_to_grid_coordinates(mobile_sensor_data)

**Compute predictions for stationary sensors**

In [3]:
# Hyperparameter search definition

cs=[0.0000002, 0.00002, 0.0002, 0.002, 0.02, 0.2, 2]
epsilons=[0.001,0.1, 0.5, 1, 2.5, 5]
# This is not needed, it's been verified it works better with epsilon_insensitive
# Also saves time
#losses=['epsilon_insensitive','squared_epsilon_insensitive']
losses=['epsilon_insensitive']
include_temp_hum = [False, True]
input_window_sizes = range(8,50, 8)

In [4]:
def test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data):
    mae = []
    mse = []
    squares = []
    pred=[]

    start_time = '2018-07-03 01:00:00'
    start_window = '2018-07-05 12:00:00'
    end_window = '2018-07-05 12:15:00'
    offset_timeint = 1

    number_of_windows = 100
    window = 15
    par_grid = functions.create_par(c, epsilon, loss)
    timeint_on_first_window = 0

    for i in range(1, number_of_windows+1):
        end_time = start_window
        # treinar sem dados moveis
        # testar em dados fixos em vez de móveis
        # Aqui rolling offset para definir a rolling mean
        timeint, par_grid = functions.train(all_static_data, None, start_time, end_time, par_grid, window, timeint_on_first_window, rolling_offset=45)
        # Usar test window start e end para fazer a previsão para daqui a 1 hora e actualizar o offset na test_mobile function call.
        test_window_start = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=45)).strftime("%Y-%m-%d %H:%M:%S")
        test_window_end = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=45)).strftime("%Y-%m-%d %H:%M:%S")
        to_test = functions.test_mobile(par_grid, timeint, all_static_data, test_window_start, test_window_end, 4)
        squares.append(to_test.shape[0])
        mae.append(mean_absolute_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        mse.append(mean_squared_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        pred.append(to_test['pred_PM2.5'].values[0])

        start_time = end_time
        start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        timeint_on_first_window = timeint+1
    return np.mean(mae), np.mean(mse)

In [5]:
results = pd.DataFrame(columns=["C", "epsilon", "loss", "temp_hum", "window_size", "mae", "mse"])
for c in cs:
    for epsilon in epsilons:
        for loss in losses:
            # for temp_hum in include_temp_hum:
                # for window_size in input_window_sizes:
                    temp_hum=False
                    window_size=0
                    mae, mse = test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data)
                    results = results.append({'C': c,
                                             'epsilon': epsilon,
                                             'loss': loss,
                                             'temp_hum': temp_hum,
                                             'window_size': window_size,
                                             'mae': mae,
                                             'mse': mse}, ignore_index=True)
                    print(mae)

0.35009226512757313
0.3522815739049064
0.4887051020474306
0.6275467973848614
1.0284053821863919
1.0284053821863919
0.20318711780468732
0.23888884948045633
0.40531366904932903
0.5965576568499693
1.0284053821863919
1.0284053821863919
0.2031868803467081
0.23888445016069265
0.40532875577176364
0.5963535126735213
1.0284053821863919
1.0284053821863919
0.2031857515152839
0.23886360355520594
0.4053984414011203
0.5954377394352067
1.0284053821863919
1.0284053821863919
0.20318109592182934
0.23877347999990323
0.40565667961082125
0.5923530985247588
1.0284053821863919
1.0284053821863919
0.20318378998647935
0.23842997501645308
0.4064782574137646
0.5818647428519881
1.0284053821863919
1.0284053821863919
0.20327280658483882
0.23767221420243648
0.40769014677768545
0.5818647428519881
1.0284053821863919
1.0284053821863919


In [6]:
results.to_csv('results/time_train1h_test1h.csv', index=False)
results

,C,epsilon,loss,temp_hum,window_size,mae,mse
0,2.000000e-07,0.001,epsilon_insensitive,False,0,0.350092,0.152798
1,2.000000e-07,0.100,epsilon_insensitive,False,0,0.352282,0.157408
2,2.000000e-07,0.500,epsilon_insensitive,False,0,0.488705,0.323140
3,2.000000e-07,1.000,epsilon_insensitive,False,0,0.627547,0.630484
4,2.000000e-07,2.500,epsilon_insensitive,False,0,1.028405,1.215368
5,2.000000e-07,5.000,epsilon_insensitive,False,0,1.028405,1.215368
6,2.000000e-05,0.001,epsilon_insensitive,False,0,0.203187,0.058700
7,2.000000e-05,0.100,epsilon_insensitive,False,0,0.238889,0.075337
8,2.000000e-05,0.500,epsilon_insensitive,False,0,0.405314,0.206562
9,2.000000e-05,1.000,epsilon_insensitive,False,0,0.596558,0.501193
